In [1]:
import os

source_path = '../bbc/'
mapping = {}
# looping through directory
for i in sorted(os.listdir(source_path)):
    # checking if it is directory or not
    if os.path.isdir(source_path+i):
        # creating the dictionary with class as key and first 300 files as key
        mapping[i] = sorted(os.listdir(source_path+i))[:50]


In [2]:
# label or class or target list
label = []

# file name
file_name = []

# text file data list
data = []
# unpacking and iterating through dictionary
for i, j in mapping.items():
    # iterating through list of files for each class
    for k in j:
        # appending labels/class/target
        label.append(i)
        file_name.append(k)
        # reading the file and appending to data list
        data.append(open(source_path+i+"/"+k, encoding="cp1252").read().replace("\n", " "))
        
print(file_name)


['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '050.txt', '001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt'

In [22]:
import pandas as pd
df = pd.DataFrame()
df["filename"] = file_name
df["label"] = label
df["document"] = data


In [9]:
import boto3
from sagemaker import get_execution_role

client = boto3.client(service_name='comprehend', region_name='us-east-1')
role = get_execution_role()


In [18]:
#key phrases
key_phrases = []
for txt_data in data:

    response_key_phrases = client.detect_key_phrases(
       Text=txt_data[:5000],
       LanguageCode='en'
    )
    key_phrases.append(list([x['Text'] for x in response_key_phrases['KeyPhrases']]))
len(key_phrases)

250

In [19]:
len(key_phrases)

250

In [24]:
#Sentiment
sentiment = []
for txt_data in data:

    response_sentiment = client.detect_sentiment(
       Text=txt_data[:5000],
       LanguageCode='en'
    )
    sentiment.append(response_sentiment['Sentiment'])


In [25]:
len(sentiment)

250

In [26]:
df['key_phrases'] = key_phrases
df['sentiment'] = sentiment
df.head()

,filename,label,document,key_phrases,sentiment
0,001.txt,business,Ad sales boost Time Warner profit Quarterly p...,"[Ad sales, Time Warner profit Quarterly profi...",NEUTRAL
1,002.txt,business,Dollar gains on Greenspan speech The dollar h...,"[Dollar gains, Greenspan speech, The dollar, i...",NEGATIVE
2,003.txt,business,Yukos unit buyer faces loan claim The owners ...,"[Yukos unit buyer, loan claim, The owners, emb...",NEUTRAL
3,004.txt,business,High fuel prices hit BA's profits British Air...,"[High fuel prices, BA, profits, British Airway...",NEUTRAL
4,005.txt,business,Pernod takeover talk lifts Domecq Shares in U...,"[Pernod takeover talk lifts, Domecq Shares, U...",NEUTRAL
